<a href="https://colab.research.google.com/github/PradipNichite/Youtube-Tutorials/blob/main/Semantic_Search_%2B_GPT_3_.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install sentence_transformers pinecone-client openai -q

In [ ]:
import pinecone
from sentence_transformers import SentenceTransformer,util
import requests
from bs4 import BeautifulSoup
model = SentenceTransformer('all-MiniLM-L6-v2') #384 dimensional

In [ ]:
model.encode("Ronaldo began his senior career with Sporting CP, before signing with Manchester United in 2003, winning the FA Cup in his first season. He would also go on to win three consecutive Premier League titles, the Champions").tolist()

In [ ]:
def get_html_content(url):
    response = requests.get(url)
    return response.content

In [ ]:
def get_plain_text(html_content):
    soup = BeautifulSoup(html_content, 'html.parser')
    for script in soup(["script"]):
        script.extract()
    return soup.get_text()

In [ ]:
def split_text_into_chunks(plain_text, max_chars=2000):
    text_chunks = []
    current_chunk = ""
    for line in plain_text.split("\n"):
        if len(current_chunk) + len(line) + 1 <= max_chars:
            current_chunk += line + " "
        else:
            text_chunks.append(current_chunk.strip())
            current_chunk = line + " "
    if current_chunk:
        text_chunks.append(current_chunk.strip())
    return text_chunks

In [ ]:
def scrape_text_from_url(url, max_chars=2000):
    html_content = get_html_content(url)
    plain_text = get_plain_text(html_content)
    text_chunks = split_text_into_chunks(plain_text, max_chars)
    return text_chunks

In [ ]:
url = input("Enter the URL to scrape text from: ")
plain_text_chunks = scrape_text_from_url(url)
print(plain_text_chunks)

In [10]:
len(plain_text_chunks)

104

In [11]:
plain_text_chunks[10]

'Raj Singh Dungarpur is credited for the selection of Tendulkar for the Indian tour of Pakistan in late 1989,[67] after one first class season.[68] The Indian selection committee had shown interest in selecting Tendulkar for the tour of the West Indies held earlier that year, but eventually did not select him, as they did not want him to be exposed to the dominant fast bowlers of the West Indies so early in his career. Tendulkar made his Test debut against Pakistan in Karachi in November 1989 aged 16 years and 205 days. He made 15 runs, being bowled by Waqar Younis, who also made his debut in that match, but was noted for how he handled numerous blows to his body at the hands of the Pakistani pace attack.[69] In the fourth and final Test in Sialkot, he was hit on the nose by a bouncer bowled by Younis, but he declined medical assistance and continued to bat even as he gushed blood from it.[70] In a 20-over exhibition game in Peshawar, held in parallel with the bilateral series, Tendulk

In [12]:
pinecone.init(api_key="api_key", environment="env") 
index = pinecone.Index("indexname")
index

In [13]:
index.describe_index_stats()

{'dimension': 384,
 'index_fullness': 0.0,
 'namespaces': {},
 'total_vector_count': 0}

In [15]:
def addData(corpusData,url):
    id  = index.describe_index_stats()['total_vector_count']
    for i in range(len(corpusData)):
        chunk=corpusData[i]
        chunkInfo=(str(id+i),
                model.encode(chunk).tolist(),
                {'title': url,'context': chunk})
        index.upsert(vectors=[chunkInfo])

In [16]:
plain_text_chunks[10]

'Raj Singh Dungarpur is credited for the selection of Tendulkar for the Indian tour of Pakistan in late 1989,[67] after one first class season.[68] The Indian selection committee had shown interest in selecting Tendulkar for the tour of the West Indies held earlier that year, but eventually did not select him, as they did not want him to be exposed to the dominant fast bowlers of the West Indies so early in his career. Tendulkar made his Test debut against Pakistan in Karachi in November 1989 aged 16 years and 205 days. He made 15 runs, being bowled by Waqar Younis, who also made his debut in that match, but was noted for how he handled numerous blows to his body at the hands of the Pakistani pace attack.[69] In the fourth and final Test in Sialkot, he was hit on the nose by a bouncer bowled by Younis, but he declined medical assistance and continued to bat even as he gushed blood from it.[70] In a 20-over exhibition game in Peshawar, held in parallel with the bilateral series, Tendulk

In [17]:
addData(plain_text_chunks,url)

In [18]:
def find_match(query,k):
    query_em = model.encode(query).tolist()
    result = index.query(query_em, top_k=k, includeMetadata=True)
    
    return [result['matches'][i]['metadata']['title'] for i in range(k)],[result['matches'][i]['metadata']['context'] for i in range(k)]

In [22]:
find_match("What awards was tendulakar awarded with?",2)

(['https://en.wikipedia.org/wiki/Sachin_Tendulkar',
  'https://en.wikipedia.org/wiki/Sachin_Tendulkar'],
 ['Tendulkar took up cricket at the age of eleven, made his Test match debut on 15 November 1989 against Pakistan in Karachi at the age of sixteen, and went on to represent Mumbai domestically and India internationally for close to twenty-four years. In 2002, halfway through his career, Wisden ranked him the second-greatest Test batsman of all time, behind Don Bradman, and the second-greatest ODI batsman of all time, behind Viv Richards.[8] Later in his career, Tendulkar was part of the Indian team that won the 2011 Cricket World Cup, his first win in six World Cup appearances for India.[9] He had previously been named "Player of the Tournament" at the 2003 edition of the tournament. Tendulkar received the Arjuna Award in 1994 for his outstanding sporting achievements, the Khel Ratna Award, India\'s highest sporting honour, in 1997, and the Padma Shri and Padma Vibhushan awards in 1

###Prompt
Answer the question as truthfully as possible using the provided context, and if the answer is not contained within the text and requires some latest information to be updated, print 'Sorry Not Sufficient context to answer query' \n"
    return header + context + "\n\n" + query + "\n"

In [23]:
import openai
openai.api_key="openai key"
def create_prompt(context,query):
    header = "Answer the question as truthfully as possible using the provided context, and if the answer is not contained within the text and requires some latest information to be updated, print 'Sorry Not Sufficient context to answer query' \n"
    return header + context + "\n\n" + query + "\n"

def generate_answer(prompt):
    response = openai.Completion.create(
    model="text-davinci-003",
    prompt=prompt,
    temperature=0,
    max_tokens=256,
    top_p=1,
    frequency_penalty=0,
    presence_penalty=0,
    stop = [' END']
    )
    return (response.choices[0].text).strip()

In [32]:
query = "Who is Sachin?"
docs,res = find_match(query,1)

In [33]:
res

['^ "Sachin Tendulkar: Who is he?". India Today. 3 April 2012. Retrieved 30 August 2021.  ^ Ghosh, Avijit (5 January 2008). "Now, Oz-speak about Brahmin dominance!". The Times of India. Retrieved 30 August 2021.  ^ Thani, L.; Mishra, R. (1999). Sensational Sachin. Diamond Pocket Books. p.\xa0113. ISBN\xa081-288-2573-9. His mother Rajni Tendulkar worked in L.I.C.  ^ "Ajit Dotes on Sachin, Who Could\'ve Guessed They\'re Half-Brothers!". The Quint. Retrieved 2 September 2018.  ^ "This man sacrificed his cricket, career for Sachin Tendulkar". Asianet Newsable.  ^ "38 special facts about Sachin Tendulkar – 3". MSN. 16 March 2012. Archived from the original on 6 June 2013. Retrieved 18 December 2012.  ^ a b "\'कांबली बहुत गुस्सैल है,\' सचिन ने KBC में अमिताभ को सुनाई थी दोस्त की कहानी; देखें Video". Jansatta (in Hindi). Retrieved 24 December 2021.  ^ "38 special facts about Sachin Tendulkar – 2". MSN. 16 March 2012. Archived from the original on 6 June 2013. Retrieved 18 December 2012.  ^ "T

In [34]:
context= "\n\n".join(res)
prompt = create_prompt(context,query)
print(prompt)

Answer the question as truthfully as possible using the provided context, and if the answer is not contained within the text and requires some latest information to be updated, print 'Sorry Not Sufficient context to answer query' 
^ "Sachin Tendulkar: Who is he?". India Today. 3 April 2012. Retrieved 30 August 2021.  ^ Ghosh, Avijit (5 January 2008). "Now, Oz-speak about Brahmin dominance!". The Times of India. Retrieved 30 August 2021.  ^ Thani, L.; Mishra, R. (1999). Sensational Sachin. Diamond Pocket Books. p. 113. ISBN 81-288-2573-9. His mother Rajni Tendulkar worked in L.I.C.  ^ "Ajit Dotes on Sachin, Who Could've Guessed They're Half-Brothers!". The Quint. Retrieved 2 September 2018.  ^ "This man sacrificed his cricket, career for Sachin Tendulkar". Asianet Newsable.  ^ "38 special facts about Sachin Tendulkar – 3". MSN. 16 March 2012. Archived from the original on 6 June 2013. Retrieved 18 December 2012.  ^ a b "'कांबली बहुत गुस्सैल है,' सचिन ने KBC में अमिताभ को सुनाई थी दोस्त 

In [35]:
reply = generate_answer(prompt)
print(reply)

Sachin Tendulkar is an Indian cricketer widely regarded as one of the greatest batsmen in the history of cricket. He is the highest run scorer of all time in International cricket. He is the only player to have scored one hundred international centuries, the first batsman to score a double century in a One Day International, and the only player to complete more than 30,000 runs in international cricket.
